In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from py2neo import Node, Relationship, Graph, NodeMatcher
import csv
import time

from req_analysis import Model, Requirement, ContextCluster

In [3]:
graph = Graph(bolt='bolt://localhost:7687/db/data', password="password")

In [4]:
# Empty graph
graph.run("MATCH (n) DETACH DELETE n")

In [5]:
node_list = []
relation_list = []
uuid_gen = iter(range(1000))

root = Node("Folder", name='root', uuid=str(next(uuid_gen)))
node_list.append(root)

req_folder = Node("Folder", name="Requirements", uuid=str(next(uuid_gen)))
node_list.append(req_folder)
struc_folder = Node("Folder", name="Structure", uuid=str(next(uuid_gen)))
node_list.append(struc_folder)
lib_folder = Node("Folder", name="Libraries", uuid=str(next(uuid_gen)))
node_list.append(lib_folder)

gs_block = Node("Block", name="Ground Station", uuid=str(next(uuid_gen)))
node_list.append(gs_block)
antenna_block = Node("Block", name="Main Antenna", uuid=str(next(uuid_gen)))
node_list.append(antenna_block)
tech_block = Node("Block", name="Technician", uuid=str(next(uuid_gen)))
node_list.append(tech_block)
antenna_part = Node("Part_Property", name="antenna", uuid=str(next(uuid_gen)))
node_list.append(antenna_part)
tech_part = Node("Part_Property", name="technician", uuid=str(next(uuid_gen)))
node_list.append(tech_part)

mission_block = Node("Block", name="Mission", uuid=str(next(uuid_gen)))
node_list.append(mission_block)
gs_part = Node("Part_Property", name="groundStation", uuid=str(next(uuid_gen)))
node_list.append(gs_part)
sc_part = Node("Part_Property", name="spacecraftVehicle", uuid=str(next(uuid_gen)))
node_list.append(sc_part)

sc_block = Node("Block", name="Spacecraft Vehicle", uuid=str(next(uuid_gen)))
node_list.append(sc_block)
pannels_block = Node("Block", name="Solar Pannel", uuid=str(next(uuid_gen)))
node_list.append(pannels_block)
computer_block = Node("Block", name="Main Computer", uuid=str(next(uuid_gen)))
node_list.append(computer_block)
pannels_part = Node("Part_Property", name="pannel", uuid=str(next(uuid_gen)))
node_list.append(pannels_part)
computer_part = Node("Part_Property", name="computer", uuid=str(next(uuid_gen)))
node_list.append(computer_part)
speed_val = Node("Value_Property", name="downlinkSpeed", uuid=str(next(uuid_gen)))
node_list.append(speed_val)

s_unit = Node("Unit", name="s", uuid=str(next(uuid_gen)))
node_list.append(s_unit)
kb_unit = Node("Unit", name="kB", uuid=str(next(uuid_gen)))
node_list.append(kb_unit)
k_unit = Node("Unit", name="K", uuid=str(next(uuid_gen)))
node_list.append(k_unit)






with open('sc_requirements.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        req = Node("Requirement", name=row[1], req_text=row[2], uuid=str(next(uuid_gen)))
        node_list.append(req)
        
        relation_list.append(Relationship(req_folder, "_", req, weight=2))
        
        allocated_node = gs_block if row[3]=="groundStation" else sc_block
#         relation_list.append(Relationship(req, "ALLOCATED", allocated_node, weight=1))
        

# Model structure (containment)
relation_list.append(Relationship(root, "_", req_folder, weight=2))
relation_list.append(Relationship(root, "_", struc_folder, weight=2))
relation_list.append(Relationship(root, "_", lib_folder, weight=2))

relation_list.append(Relationship(struc_folder, "_", gs_block, weight=2))
relation_list.append(Relationship(struc_folder, "_", mission_block, weight=2))
relation_list.append(Relationship(struc_folder, "_", sc_block, weight=2))

relation_list.append(Relationship(lib_folder, "_", s_unit, weight=2))
relation_list.append(Relationship(lib_folder, "_", kb_unit, weight=2))
relation_list.append(Relationship(lib_folder, "_", k_unit, weight=2))

relation_list.append(Relationship(mission_block, "ATTRIBUTE", gs_part, weight=1))
relation_list.append(Relationship(mission_block, "ATTRIBUTE", sc_part, weight=1))
relation_list.append(Relationship(gs_part, "TYPED", gs_block, weight=1))
relation_list.append(Relationship(sc_part, "TYPED", sc_block, weight=1))

relation_list.append(Relationship(gs_block, "_", antenna_block, weight=2))
relation_list.append(Relationship(gs_block, "_", tech_block, weight=2))
relation_list.append(Relationship(gs_block, "ATTRIBUTE", antenna_part, weight=1))
relation_list.append(Relationship(gs_block, "ATTRIBUTE", tech_part, weight=1))
relation_list.append(Relationship(antenna_part, "TYPED", antenna_block, weight=1))
relation_list.append(Relationship(tech_part, "TYPED", tech_block, weight=1))

relation_list.append(Relationship(sc_block, "_", pannels_block, weight=2))
relation_list.append(Relationship(sc_block, "_", computer_block, weight=2))
relation_list.append(Relationship(sc_block, "ATTRIBUTE", pannels_part, weight=1))
relation_list.append(Relationship(sc_block, "ATTRIBUTE", computer_part, weight=1))
relation_list.append(Relationship(pannels_part, "TYPED", pannels_block, weight=1))
relation_list.append(Relationship(computer_part, "TYPED", computer_block, weight=1))
relation_list.append(Relationship(computer_block, "ATTRIBUTE", speed_val, weight=1))
relation_list.append(Relationship(speed_val, "TYPED", kb_unit, weight=1))

In [6]:
for node in node_list:
    graph.create(node)
for relation in relation_list:
    graph.create(relation)

## Workflow

In [7]:
model = Model(graph, 0.045)

## Loop to match all the requirements

In [8]:
time1 = time.time()
matcher = NodeMatcher(graph)
nodematcher = matcher.match("Requirement")

req_object_list=[]

for req_node in nodematcher:
    time2 = time.time()
    
    req_object = model.create_req_object(req_node)
    req_object.init_req_nodes()
    
    allocated_context = req_object.context_cluster_mapping()
    
    if allocated_context!=None:
        rel = Relationship(req_object.node, "ALLOCATED", allocated_context.source_node, weight=1)
        graph.create(rel)
        print('\nREQ TEXT:\n\t', req_node['req_text'])
        print('ALLOCATED NODE:\n\t---', allocated_context.source_node['name'], '\nALLOCATED NODE ATTRIBUTES: ')
        for el in allocated_context.context:
            print('\t-', el['name'])
        print('\nExecution time: ', time.time()-time2)
        print('___________________\n')
        

    req_object.match_req_tokens(pos_list=["NOUN", "PROPN", "NUM"], relationship_filter='ATTRIBUTE|TYPED|ALLOCATED>|_', label_filter='-ReqPhrase', min_level=1, max_level=8, weighted_depth_limit=8)
    
    model.append_req_graph(req_object)
    model.link_matches(req_object.transclusion_relations)
    
    req_object_list.append(req_object)
print("\nTotal matching execution time: ", time.time()-time1)

... CONTEXT MATCH:  The spacecraft Spacecraft Vehicle (_288:Block {name: 'Mission', uuid: '9'})
... SOURCE NODE MATCH:  The spacecraft Spacecraft Vehicle
... CONTEXT MATCH:  The spacecraft Spacecraft Vehicle (_292:Block {name: 'Solar Pannel', uuid: '13'})
... CONTEXT MATCH:  The spacecraft Spacecraft Vehicle (_293:Block {name: 'Main Computer', uuid: '14'})

REQ TEXT:
	 The spacecraft shall be able to transmit 100 kB in less than 300s.
ALLOCATED NODE:
	--- Spacecraft Vehicle 
ALLOCATED NODE ATTRIBUTES: 
	- Main Computer
	- Solar Pannel
	- Mission

Execution time:  0.033242225646972656
___________________

... CONTEXT MATCH:  The main antenna Main Antenna (_283:Block {name: 'Ground Station', uuid: '4'})
... SOURCE NODE MATCH:  the ground station Ground Station
... SOURCE NODE MATCH:  The main antenna Main Antenna
... CONTEXT MATCH:  the ground station Ground Station (_284:Block {name: 'Main Antenna', uuid: '5'})
... CONTEXT MATCH:  the ground station Ground Station (_285:Block {name: 'Te

In [9]:
for req_object in req_object_list:
    req_object.init_transcluded_text()
    print(req_object.name, "--->", req_object.transcluded_text)


Data Transmission ---> <cref id='root::Structure::Spacecraft Vehicle'>The spacecraft</cref> shall be able to transmit 100 <cref id='root::Libraries::kB'>kB</cref> in less than 300s .

Antenna Orientation ---> <cref id='root::Structure::Ground Station::antenna'>The main antenna</cref> of <cref id='root::Structure::Ground Station'>the ground station</cref> shall be oriented towards <cref id='root::Structure::Mission::spacecraftVehicle'>the spacecraft vehicle</cref> at least 8 hours per sol .

Solar Pannel Exposure ---> <cref id='root::Requirements::Solar Pannel Exposure::Solar Pannel'>The solar pannels</cref> shall be exposed to the sun at least 40% of a given sol .

Main Computer Temperature ---> The temperature of <cref id='root::Requirements::Main Computer Temperature::Main Computer'>the main computer</cref> shall be comprised between 300 K and 370 K.

Main Computer Storage ---> The main computer storage capacity shall not exceed 90% at any given time .

Ground Station Population ---

# Queries

## Isolate a requirement

match (n) where (n:Unit or n:Folder or n:Block or n:Part_Property)
match path=((r:Requirement {name: 'Antenna Orientation'})-[:IS]->()-[:IS_COMPOSED_OF]->())
return n, r, nodes(path)

#### apoc
MATCH (r:Requirement {name: 'Antenna Orientation'}) 
CALL apoc.path.spanningTree(r, {relationshipFilter:"PART|_|ALLOCATED>|TYPED>", minLevel:1, maxLevel:3}) yield path return path

In [10]:
cursor = graph.run("""MATCH (n:Requirement {name: 'Antenna Orientation'})
CALL apoc.path.spanningTree(n, {relationshipFilter:'ALLOCATED>|TYPED|PART|_', labelFilter:'', minLevel:1, maxLevel:6}) YIELD path
WITH last(nodes(path)) as node, reduce(weight = 0, rel IN relationships(path) | weight+rel.weight) as depth
WHERE depth<7
WITH depth, collect(node) as nodes_at_depth
ORDER BY depth ASC
RETURN nodes_at_depth, depth""")